In [1]:
import pandas as pd
train = pd.read_parquet("data/train-00000-of-00001.parquet")
train["dagw_domain"].unique()

array(['News', 'Web', 'Conversation', 'Social Media', 'Wiki & Books',
       'Legal', None, 'dannet'], dtype=object)

In [118]:
chosen_sent = train.iloc[28]
print(chosen_sent["text"])
print()
for ent in chosen_sent["ents"]:
    print(chosen_sent["text"][ent["start"]:ent["end"]], ":", ent["label"])



Den franske Revolutions Historie i kort Begreb

franske : NORP


In [129]:
import torch
import sys
import myutils
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

# set seed for consistency
torch.manual_seed(8446)
# Set some constants
MLM = 'vesteinn/DanskBERT'

BATCH_SIZE = 32
LEARNING_RATE = 0.00001
EPOCHS = 3

UNK = "[UNK]"
MAX_TRAIN_SENTS=500
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"




In [150]:
tokenizer = AutoTokenizer.from_pretrained(MLM)
chosen_sent = train.iloc[8]

print(tokenizer.tokenize(tokenizer.decode(tokenizer.encode(chosen_sent["text"]))))
print(chosen_sent["text"])
print()
for ent in chosen_sent["ents"]:
    tokens = chosen_sent["text"][ent["start"]:ent["end"]], ":", ent["label"]
    labels = [''] * len(tokens)
    print(tokenizer.tokenize(tokenizer.decode(tokenizer.encode(chosen_sent["text"][:ent["start"]]))))








['<s>', '▁Hand', '▁mod', '▁Hand', ',', '▁18', '16', '▁', '</s>']
Hand mod Hand, 1816


['<s>', '</s>']
['<s>', '▁Hand', '▁mod', '▁Hand', ',', '▁', '</s>']


In [237]:
tokenizer(chosen_sent["text"]).tokens()

['<s>',
 '▁Taler',
 '▁9:',
 '▁også',
 '▁ret',
 '▁meget',
 '▁pas',
 ',',
 '▁men',
 '▁mest',
 '▁på',
 '▁1',
 '▁',
 '</s>']

In [151]:

def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [236]:
chosen_sent = train.iloc[1231]
print(chosen_sent["text"])

for token in chosen_sent["tokens"]:
    print(chosen_sent["text"][token["start"]:token["end"]])




Taler 9: også ret meget pas, men mest på 1

Taler
9
:
også
ret
meget
pas
,
men
mest
på
1




In [ ]:
tokenize_and_preserve_labels(chosen_sent["text"], )